In [4]:
import psycopg2
import pandas as pd

In [5]:
df = pd.read_csv('all_sources_geoed.csv', index_col=[0], sep='|')

In [6]:
def copy_query(c, file_path, table_name, cols_tuple):

    schema_table = table_name
    with open(file_path, 'r') as f:
        next(f)
        c.copy_from(f, schema_table, sep=',', null='null', columns=cols_tuple)

In [7]:
file_p = "all_sources_geoed.csv"
table_n = "original_all_sources_geoed"
cols_t = ('ind','src', 'state', 'city', 'description', 'tags', 'geolocation', 'title', 'date', 'date_text', 'id', 'lat', 'long')

In [8]:
try:
    #function for establishing connection
    connection = psycopg2.connect(user= "postgres_hrf_L28",
                                        password= "P98bOOSz4YWnojw1htOo", 
                                        host= "database-2-hrf-ds-e.cjb51vwbigfl.us-east-1.rds.amazonaws.com",
                                        port = "5432",  
                                        database = "postgres")
    cursor = connection.cursor()
    #print POSTGRES Conn properties
    print(connection.get_dsn_parameters(), "\n")

except Exception as ex:
        print("connection error: ")
        print (ex)

{'user': 'postgres_hrf_L28', 'dbname': 'postgres', 'host': 'database-2-hrf-ds-e.cjb51vwbigfl.us-east-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [9]:
#THIS line of code was quickly written to remove all commas from strings for ease of SQL query,
#can be refined to not brute force delete all commas

#df['description'] = df['description'].str.replace(",", "")

In [10]:
df.columns

Index(['src', 'state', 'city', 'description', 'tags', 'geolocation', 'title',
       'date', 'date_text', 'id', 'lat', 'long'],
      dtype='object')

In [23]:
#THIS NEEDS TO RUN THROUGH WHOLE DF NOT JUST SRC

cols_to_check = ['src', 'state', 'city', 'description', 'tags', 'geolocation', 'title',
       'date', 'date_text', 'id', 'lat', 'long']

df['src'] = df['src'].astype(str).str.replace('|', '', regex=True)

df.iloc[88]
df.to_csv('all_sources_geoed.csv', sep='|')

In [12]:
#create all_sources_geod.csv
cursor.execute("""CREATE TABLE all_sources_geod(
    ind SERIAL PRIMARY KEY,
    src text,
    state text,
    city text,
    description text,
    tags text,
    geolocation text,
    title text,
    date text,
    date_text text,
    id text,
    lat float,
    long float)""")


connection.commit()


DuplicateTable: relation "all_sources_geod" already exists


In [7]:
try:
    #INSERTING
    copy_query(cursor, file_p, table_n, cols_t)
    #INSERTING END

    #COMMIT
    connection.commit()
    #COMMITING END
except (Exception, psycopg2.Error) as error:
    print("error while connecting to postgreSQL", error)


error while connecting to postgreSQL current transaction is aborted, commands ignored until end of transaction block



In [29]:
#TEST QUERY :)

sql="""
SELECT *
FROM all_sources_geod
ORDER BY ind
"""
pd.read_sql(sql, con=connection)

,ind,src,state,city,description,tags,geolocation,title,date,date_text,id,lat,long
0,0,['https://www.youtube.com/watch?v=s7MM1VauRHo'],Washington,Olympia,Footage shows a few individuals break off from a protest to smash City Hall windows. Protesters shout at vandals to stop. Police then arrive. They arrest multiple individuals near the City Hall windows including one individual who appeared to approach the vandals in an effort to defuse the situation. Police fire tear gas and riot rounds at protesters during the arrests. Protesters become agitated. After police walk arrestee away protesters continue to shout at police. Police respond with a second bout of tear gas and riot rounds. A racial slur can be heard shouted although it is unsure who is shouting.,"['arrest', 'less-lethal', 'projectile', 'protester', 'shoot', 'tear-gas']","{'lat': '47.0378741', 'long': '-122.9006951'}",Police respond to broken windows with excessive force,2020-05-31,May 31st,wa-olympia-1,47.037874,-122.900695
1,1,['https://mobile.twitter.com/chadloder/status/1267011092045115392'],Washington,Seattle,"""Officer pins protester with his knee on his neck. His partner intervenes and moves his knee onto the individual's back. Possibly related to OPD Case 2020OPA-0324 - """"Placing the knee on the neck area of two people who had been arrested""""""","['arrest', 'knee-on-neck', 'protester']","{'lat': '47.6062095', 'long': '-122.3320708'}",Officer pins protester by pushing his knee into his neck,2020-05-30,May 30th,wa-seattle-1,47.606209,-122.332071
2,2,['https://twitter.com/gunduzbaba1905/status/1266937500607614982'],Washington,Seattle,A couple of police officers are seen beating and manhandling an unarmed man. The officers are throwing punches while he was on the ground and pinned. Related to Seattle OPA Case 2020OPA-0330.,"['beat', 'protester', 'punch']","{'lat': '47.6062095', 'long': '-122.3320708'}",Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2,47.606209,-122.332071
3,3,['https://www.reddit.com/r/Bad_Cop_No_Donut/comments/gtt3w8/i_caught_the_moment_seattle_police_pepper_sprayed/'],Washington,Seattle,A police officer randomly and indiscriminately pepper sprays civilians walking around which then went on to spark a riot.,"['less-lethal', 'pepper-spray', 'protester']","{'lat': '47.6062095', 'long': '-122.3320708'}",Police indiscriminately pepper spray peaceful protesters,2020-05-31,May 31st,wa-seattle-3,47.606209,-122.332071
4,4,['https://www.fox10phoenix.com/news/video-shows-milk-poured-over-face-of-child-pepper-sprayed-in-seattle-protest'],Washington,Seattle,Police pepper sprays a young child who is seen crying while protesters pour milk on the distressed girl. Related to Seattle OPA Case 2020OPA-0322.,"['child', 'inhumane-treatment', 'less-lethal', 'pepper-spray', 'protester']","{'lat': '47.6062095', 'long': '-122.3320708'}",Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4,47.606209,-122.332071
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,1173,"['https://www.pinknews.co.uk/2020/06/05/police-raid-blazing-saddle-des-moines-iowa-black-lives-matter-stonewall/?fbclid=IwAR20SUPSke3O_U2jqXaicl3dOcKwKPYlG5eOc131oao76Z2rEXXz0-ASORQ', 'http://iowainformer.com/politics/2020/06/des-moines-gay-bar-raided-amid-black-lives-matter-protests-as-pride-month-begins/', 'https://twitter.com/aaronpcalvin/status/1268708285517004801', 'https://www.google.ca/maps/place/The+Blazing+Saddle/@41.590306,-93.6114412,3a,75y,275.31h,90t/data=!3m7!1e1!3m5!1sysYseTIG29m9bjNYmZ71zQ!2e0!6s%2F%2Fgeo3.ggpht.com%2Fcbk%3Fpanoid%3DysYseTIG29m9bjNYmZ71zQ%26output%3Dthumbnail%26cb_client%3Dmaps_sv.tactile.gps%26thumb%3D2%26w%3D203%26h%3D100%26yaw%3D275.31018%26pitch%3D0%26thumbfov%3D100!7i16384!8i8192!4m7!3m6!1s0x87ee99b1e2f004c9:0xf6f27ceb30ff9e27!8m2!3d41.5902914!4d-93.6115743!14m1!1BCgIgARICCAI', 'https://www.facebook.com/permalink.php?story_fbid=1182620968746799&id=100009967212544']",Iowa,Des Moines,"""""""Des Moines police approached The Blazi

In [26]:
df.iloc[195]

src                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   ['https://www.theroot.com/a-letter-not-from-a-birmingham-jail-1843926346', 'https://www.al.com/news/birmingham/2020/06/journalist-michael-harriot-arrested-after-birmingham-protest.html', 'https://www.youtube.com/watch?v=H308RU7jtM4', 'https://www.poynter.org/reporting-editing/2020/23-guidelines-for-journalists-to-safely-cover-protests-t

In [27]:
connection.commit()
with open('all_sources_geoed.csv', 'r', encoding="utf8") as row:
    next(row) #skip header
    cursor.copy_from(row, 'all_sources_geod', sep='|')


connection.commit()